# Download audio file


In [44]:
import youtube_dl
#from youtube_dl import YoutubeDL
import re
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

In [42]:
audio_directory=os.path.abspath(os.path.join(os.getcwd(),os.pardir,os.pardir))+'/data/Audio/'
audio_directory='C:/Users/Goller/Desktop/audio-event-tagging/data/Audio/'


## Read file from the csv file

## Create a pandas dataset

In [35]:
df=pd.read_csv(os.path.abspath(os.path.join(os.getcwd(),os.pardir,os.pardir))+'/data/data_split.csv',index_col=0)
df.head()  

,url,start_time,end_time,class_label
0,SpNYhh5nwBk,50.0,60.0,Animal
1,Y-5QMgGV1H0,0.0,10.0,Animal
2,2yRlkPqa6rQ,20.0,30.0,Animal
3,NYiIU62GHV4,30.0,40.0,Animal
4,x3c0szZ7VYI,60.0,70.0,Animal


In [36]:
df['class_label'].value_counts()

Natural    5000
Humans     5000
Animal     5000
Name: class_label, dtype: int64

## Download Audio


In [48]:
    
for i, row in tqdm(df.iterrows()):
    #url = "'https://www.youtube.com/embed/" + row['url'] + "'"
    file_name = str(i)+"_"+row['class_label']
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '320',
        }],
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([row['url']])
    if i== 5:
        break

0it [00:00, ?it/s]

[youtube] SpNYhh5nwBk: Downloading webpage
[download] Destination: ROO-fest-SpNYhh5nwBk.webm
[download] 100% of 828.58KiB in 00:00                   


ERROR: ffprobe/avprobe and ffmpeg/avconv not found. Please install one.
0it [00:01, ?it/s]


DownloadError: ERROR: ffprobe/avprobe and ffmpeg/avconv not found. Please install one.

In [43]:
for i, row in tqdm(df.iterrows()):
    url = "'https://www.youtube.com/embed/" + row['url'] + "'"
    file_name = str(i)+"_"+row['class_label']
    
    try:
        command_1 = "ffmpeg -ss " + str(row['start_time']) + " -i $(youtube-dl -f 140 --get-url " +\
                    url + ") -t 10 -c:v copy -c:a copy " + file_name + ".mp4"

        command_2 = "ffmpeg -i "+ file_name +".mp4 -vn -acodec pcm_s16le -ar 44100 -ac 1 " + audio_directory + file_name + ".wav"

        command_3 = 'rm ' + file_name + '.mp4' 

        # Run the 3 commands
        os.system(command_1 + ';' + command_2 + ';' + command_3 + ';')
    
    except:
        print(i, url)
        pass

15000it [03:42, 67.33it/s]


In [41]:
os.getcwd()

'C:\\Users\\Goller\\Desktop\\audio-event-tagging\\src\\utils'

citare: https://github.com/HareeshBahuleyan/music-genre-classification/blob/master/1_audio_retrieval.ipynb